In [ ]:
##########################################################################################################################################
######################################## The following script averages the 10 model realizations  ########################################
##########################################################################################################################################

In [1]:
import ee # Import the Python API package
import os
import geemap # The geemap Python package is built upon the ipyleaflet and folium packages and implements several methods for interacting with Earth Engine data layer
ee.Authenticate() # Authenticate to the Earth Engine servers
ee.Initialize() # Initialize the API

Enter verification code:  4/1AX4XfWjZOlRTvrkLacCSToY-rgJyydoUzccTT998ZgDdN--CBymQZPOA5c4



Successfully saved authorization token.


In [2]:
# Load the image collection containinig models
SoilPhototrophsProductivityInLog = ee.ImageCollection('users/romainwalcker/SoilPhototrophsV6')

In [3]:
# Create a function to convert log NPP values to linear NPP values
def LogConvert(image): 
  return image.exp().subtract(1)
# Map the function
SoilPhototrophsProductivity = SoilPhototrophsProductivityInLog.map(LogConvert)

In [4]:
# Average models
meanModel = SoilPhototrophsProductivity.toBands().reduce(ee.Reducer.mean()).rename('Mean gC/m²/yr')
stdModel = SoilPhototrophsProductivity.toBands().reduce(ee.Reducer.stdDev())
varCoefModel = stdModel.divide(meanModel).rename('VarCoef')

In [5]:
# Create a new map
Map1 = geemap.Map(center=[0,0], zoom=2)
palette = ['330044', '220066', '1133cc', '33dd00', 'ffda21', 'ff6622', 'd10000']
Map1.addLayer(meanModel, {'min': 0, 'max': 70, 'palette': palette},'Mean predicted NPP')
Map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [13]:
# Create a new map
Map2 = geemap.Map(center=[0,0], zoom=2)
Map2.addLayer(varCoefModel, {'min': 0, 'max': 0.7},'Coefficient of variation')
Map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [12]:
# Create a bounding rectangle for the entire planet
unboundedGeo = ee.Geometry.Rectangle([-180, -88, 180, 88], "EPSG:4326", False)
# Create en empty imageCollection to store maps in your Asset folder
ee.data.createAsset({'type': 'ImageCollection'}, 'users/romainwalcker/SoilPhototrophsV6Final')
# Export to your Asset
taskParams = {'assetId': 'users/romainwalcker/SoilPhototrophsV6Final','region': unboundedGeo,'crs':'EPSG:4326','crsTransform':[0.008333333333333333,0,-180,0,-0.008333333333333333,90],'maxPixels': 1e12}
task = ee.batch.Export.image.toAsset(meanModel,str('MeanNPP'),**taskParams)
task.start()
taskParams = {'assetId': 'users/romainwalcker/SoilPhototrophsV6Final','region': unboundedGeo,'crs':'EPSG:4326','crsTransform':[0.008333333333333333,0,-180,0,-0.008333333333333333,90],'maxPixels': 1e12}
task = ee.batch.Export.image.toAsset(varCoefModel,str('VarCoef'),**taskParams)
task.start()
# Export to your Drive
#taskParams = {'crs': 'EPSG:4326','crsTransform':[0.008333333333333333,0,-180,0,-0.008333333333333333,90],'maxPixels': 10e15,'fileFormat': 'GeoTIFF',
              #'skipEmptyTiles': True,'region': unboundedGeo ,'folder': 'Jupyter'}
#task = ee.batch.Export.image.toDrive(meanModel,str('MeanNPP'),**taskParams)
#task.start()
#taskParams = {'crs': 'EPSG:4326','crsTransform':[0.008333333333333333,0,-180,0,-0.008333333333333333,90],'maxPixels': 10e15,'fileFormat': 'GeoTIFF',
              #'skipEmptyTiles': True,'region': unboundedGeo ,'folder': 'Jupyter'}
#task = ee.batch.Export.image.toDrive(varCoefModel,str('VarCoefNPP'),**taskParams)
#task.start()